# Clasificación multiclase

Ya hemos construido nuestro primer modelo de clasificación binario. Comenzamos por una sola variable independiente, introducimos varias, y ahora haremos una extensión importante: **clasificación con más de dos categorías**. Un ejemplo de esto podría ser el aprender una función que, con las coordenadas de latitud y longitud de un punto geográfico dentro del territorio nacional, nos devuelve una de las siguientes:

- Una etiqueta $\texttt{provincia} \in \left\{\texttt{San Jose}, \texttt{Alajuela}, \texttt{Cartago}, \texttt{Heredia}, \texttt{Guanacaste}, \texttt{Puntarenas}, \texttt{Limon} \right\}$. O, en su defecto,
- Un vector $\mathbf{y}$ con siete valores que denotan la probabilidad de que el punto indicado se encuentre en una provincia data, de modo que $\sum_{i=1}^{7} y_i = 1$.


### El dataset de los lirios

Para este notebook, haremos uso de uno de los datasets más famosos y antiguos de la historia de la estadística: los datos de los lirios de Fisher, conocido en inglés como el *Iris dataset*. Este dataset contiene cincuenta muestras de mediciones de lirios, con cinco columnas: la longitud y ancho del pétalo y del sépalo, respectivamente, y una etiqueta que indica la especie de lirio al que corresponde, que puede ser *versicolor*, *virginica* o *setosa*. Nuestro objetivo, entonces, es aprender la función $f\left(\texttt{l}\_\texttt{petalo}, \texttt{a}\_\texttt{petalo}, \texttt{l}\_\texttt{sepalo}, \texttt{a}\_\texttt{sepalo} \right) \mapsto \left[ p\left(\texttt{versicolor}\right), p\left(\texttt{virginica} \right), p\left(\texttt{setosa} \right)\right]$.


## Clasificación uno-contra-todos

## Clasificación Softmax